<a href="https://colab.research.google.com/github/miladqolamee/NeuroMatch_2022_group_project/blob/main/Coding_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is created for exploratory data analysis on Allen 2P dataset by \name of our group\ 

---
# First inatal and upgrade allensdk package

In [ ]:
# @title Instal allensdk package
# @markdown Run this cell just one time.
# @markdown After running this cell, restart the Runtime

pip install allensdk

In [ ]:
# @title Upgrade the package
!pip install --upgrade pip
!pip install allensdk


In [ ]:
# @title Improt relevant packages
import numpy as np
import pandas as pd

import allensdk
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

---
# A brief summary on Allen 2P dataset 


- `First`
-
-
For more information see:
[Link](https://allensdk.readthedocs.io/en/latest/visual_behavior_optical_physiology.html).